In [1]:
import yfinance as yf

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import math

# import Model
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, TimeDistributed
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError

from tensorflow.keras.utils import plot_model

from sklearn.metrics import explained_variance_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ttest_ind

In [2]:
########### 4 saham

In [3]:
# Load data
LQ45 = yf.download("^JKLQ45", start= "2007-11-13", end="2024-05-20")
ICBP = yf.download("ICBP.JK", start= "2007-11-13", end="2024-05-20")
BBCA = yf.download("BBCA.JK", start= "2007-11-13", end="2024-05-20")
JSMR = yf.download("JSMR.JK", start= "2007-11-13", end="2024-05-20")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
LQ45.head

<bound method NDFrame.head of                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2007-11-13  588.679993  590.719971  578.190002  584.729980  584.729980   
2007-11-14  596.700012  598.979980  589.659973  591.690002  591.690002   
2007-11-15  592.669983  598.340027  588.440002  594.840027  594.840027   
2007-11-16  585.729980  590.729980  579.500000  585.340027  585.340027   
2007-11-19  588.530029  589.950012  577.729980  580.169983  580.169983   
...                ...         ...         ...         ...         ...   
2024-05-13  887.549988  897.210022  885.359985  895.549988  895.549988   
2024-05-14  895.200012  898.780029  891.010010  892.580017  892.580017   
2024-05-15  893.020020  904.780029  890.919983  901.400024  901.400024   
2024-05-16  906.669983  912.590027  904.900024  909.309998  909.309998   
2024-05-17  909.909973  925.640015  909.030029  919.510010  919.510010   

       

In [5]:
# Substitute 0 adj close with previous day's volume
LQ45['Adj Close'] = LQ45['Adj Close'].replace(0, method='ffill')
ICBP['Adj Close'] = ICBP['Adj Close'].replace(0, method='ffill')
BBCA['Adj Close'] = BBCA['Adj Close'].replace(0, method='ffill')
JSMR['Adj Close'] = JSMR['Adj Close'].replace(0, method='ffill')

In [6]:
# get returns from adj_close
adj_close_LQ = LQ45.filter(['Adj Close'])
returns_LQ = adj_close_LQ.pct_change().fillna(0)

adj_close_ICBP = ICBP.filter(['Adj Close'])
returns_ICBP = adj_close_ICBP.pct_change().fillna(0)

adj_close_BBCA = BBCA.filter(['Adj Close'])
returns_BBCA = adj_close_BBCA.pct_change().fillna(0)

adj_close_JSMR = JSMR.filter(['Adj Close'])
returns_JSMR = adj_close_JSMR.pct_change().fillna(0)

In [7]:
# Resample daily returns to weekly returns
LQ45_returns_weekly = returns_LQ.resample('W').sum()
ICBP_returns_weekly = returns_ICBP.resample('W').sum()
BBCA_returns_weekly = returns_BBCA.resample('W').sum()
JSMR_returns_weekly = returns_JSMR.resample('W').sum()

In [8]:
LQ45_returns_weekly.head

<bound method NDFrame.head of             Adj Close
Date                 
2007-11-18   0.001256
2007-11-25  -0.037405
2007-12-02   0.049754
2007-12-09   0.035262
2007-12-16  -0.016101
...               ...
2024-04-21  -0.045467
2024-04-28  -0.023241
2024-05-05   0.005772
2024-05-12  -0.010960
2024-05-19   0.028931

[862 rows x 1 columns]>

In [9]:
ICBP_returns_weekly.head

<bound method NDFrame.head of             Adj Close
Date                 
2007-11-18   0.046219
2007-11-25  -0.098523
2007-12-02   0.002168
2007-12-09   0.021978
2007-12-16  -0.008628
...               ...
2024-04-21  -0.106229
2024-04-28   0.087907
2024-05-05   0.024829
2024-05-12  -0.006912
2024-05-19  -0.019680

[862 rows x 1 columns]>

In [10]:
BBCA_returns_weekly.head

<bound method NDFrame.head of             Adj Close
Date                 
2007-11-18  -0.006444
2007-11-25  -0.019860
2007-12-02   0.052017
2007-12-09   0.055424
2007-12-16  -0.012937
...               ...
2024-04-21  -0.035596
2024-04-28   0.017117
2024-05-05   0.024085
2024-05-12  -0.048785
2024-05-19   0.039540

[862 rows x 1 columns]>

In [11]:
JSMR_returns_weekly.head

<bound method NDFrame.head of             Adj Close
Date                 
2007-11-18  -0.012346
2007-11-25  -0.075587
2007-12-02   0.044101
2007-12-09   0.024028
2007-12-16  -0.022850
...               ...
2024-04-21  -0.012778
2024-04-28  -0.070677
2024-05-05  -0.002065
2024-05-12   0.038350
2024-05-19  -0.038764

[862 rows x 1 columns]>

In [13]:
# Combine the two DataFrames
weekly_returns_4stocks = pd.concat([LQ45_returns_weekly, ICBP_returns_weekly, BBCA_returns_weekly, JSMR_returns_weekly], axis=1)
weekly_returns_4stocks.columns = ['LQ45 Returns', 'ICBP Returns', 'BBCA returns','JSMR returns']

# Save the combined DataFrame to a CSV file
weekly_returns_4stocks.to_csv('../out/weekly_returns_4stocks.csv', index=True)  # index=True to include the date index in the CSV

print(f"Weekly returns combined and saved to {weekly_returns_4stocks}")

Weekly returns combined and saved to             LQ45 Returns  ICBP Returns  BBCA returns  JSMR returns
Date                                                              
2007-11-18      0.001256      0.046219     -0.006444     -0.012346
2007-11-25     -0.037405     -0.098523     -0.019860     -0.075587
2007-12-02      0.049754      0.002168      0.052017      0.044101
2007-12-09      0.035262      0.021978      0.055424      0.024028
2007-12-16     -0.016101     -0.008628     -0.012937     -0.022850
...                  ...           ...           ...           ...
2024-04-21     -0.045467     -0.106229     -0.035596     -0.012778
2024-04-28     -0.023241      0.087907      0.017117     -0.070677
2024-05-05      0.005772      0.024829      0.024085     -0.002065
2024-05-12     -0.010960     -0.006912     -0.048785      0.038350
2024-05-19      0.028931     -0.019680      0.039540     -0.038764

[862 rows x 4 columns]
